In [ ]:
from langchain.agents import create_sql_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
import warnings

# 경고 메시지 숨기기
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

llm = ChatOpenAI(temperature=0.1, model="gpt-4.1-nano")

db = SQLDatabase.from_uri("sqlite:///movies.sqlite")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,  # ZERO_SHOT_REACT_DESCRIPTION 대신 OPENAI_FUNCTIONS 사용
    handle_parsing_errors=True,
    max_iterations=5,  # 최대 반복 횟수 제한
)

# 더 구체적인 프롬프트 사용
result = agent.invoke({
    "input": "Find the top 5 directors with the highest total gross earnings from their films. Please provide the director names and their total gross earnings."
})

print(result["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


directors, movies
Invoking: `sql_db_schema` with `{'table_names': 'directors,movies'}`



CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	A